In [1]:
import pandas as pd
from collections import Counter
from itertools import combinations

In [2]:
factors = pd.read_csv('factors.csv')
factors.head()

,order_id,product_id,product_name,department_id,department_name
0,1,49302,Bulgarian Yogurt,16,dairy eggs
1,1,11109,Organic 4% Milk Fat Whole Milk Cottage Cheese,16,dairy eggs
2,1,10246,Organic Celery Hearts,4,produce
3,1,49683,Cucumber Kirby,4,produce
4,1,43633,Lightly Smoked Sardines in Olive Oil,15,canned goods


In [3]:
factors.isna().sum()

order_id           0
product_id         0
product_name       0
department_id      0
department_name    0
dtype: int64

# Part A1: Products Report

In [4]:
product_id_to_name = factors.drop_duplicates(subset='product_id')[['product_id', 'product_name']].set_index('product_id')['product_name'].to_dict()
order_df = factors.groupby('order_id')['product_id'].apply(list).reset_index(name='order')

In [5]:
item_counts = Counter()
pair_counts = Counter()
for products in order_df['order']:
    unique_items = set(products)
    item_counts.update(unique_items)
    pair_counts.update(combinations(sorted(unique_items), 2))

In [6]:
num_orders = len(order_df)
results = []
for (item1, item2), count_pair in pair_counts.items():
    support = count_pair / num_orders
    confidence_1_to_2 = count_pair / item_counts[item1]
    confidence_2_to_1 = count_pair / item_counts[item2]

    if support >= 0.003:
        if confidence_1_to_2 >= 0.25:
            results.append((item1, item2, support, confidence_1_to_2))
        if confidence_2_to_1 >= 0.25:
            results.append((item2, item1, support, confidence_2_to_1))


In [7]:
rules_df = pd.DataFrame(results, columns=['product_id_1', 'product_id_2', 'support', 'confidence'])
rules_df['product_name_1'] = rules_df['product_id_1'].map(product_id_to_name)
rules_df['product_name_2'] = rules_df['product_id_2'].map(product_id_to_name)
rules_df = rules_df[['product_id_1', 'product_id_2', 'product_name_1', 'product_name_2', 'confidence', 'support']]
rules_df.sort_values(by='product_id_1', ascending=False)

,product_id_1,product_id_2,product_name_1,product_name_2,confidence,support
44,49683,24852,Cucumber Kirby,Banana,0.307915,0.005663
19,47766,24852,Organic Avocado,Banana,0.299096,0.016889
35,47626,24852,Large Lemon,Banana,0.265274,0.016447
1,47209,13176,Organic Hass Avocado,Bag of Organic Bananas,0.331825,0.018444
6,47144,24852,Unsweetened Original Almond Breeze Almond Milk,Banana,0.298294,0.003331
47,46906,24852,Grape White/Green Seedless,Banana,0.263261,0.003064
8,45066,24852,Honeycrisp Apple,Banana,0.346663,0.009382
16,43961,24852,Organic Peeled Whole Baby Carrots,Banana,0.262602,0.004923
48,41950,13176,Organic Tomato Cluster,Bag of Organic Bananas,0.250197,0.004840
15,41787,24852,Bartlett Pears,Banana,0.386081,0.003552


In [8]:
rules_df.to_csv('products_report.csv', index=False)

# Part A2: Departments Report

In [9]:
dept_id_to_name = factors.drop_duplicates(subset='department_id')[['department_id', 'department_name']].set_index('department_id')['department_name'].to_dict()
order_dept_df = factors.groupby('order_id')['department_id'].apply(lambda x: list(set(x))).reset_index(name='departments')
num_orders = len(order_dept_df)
dept_counts = Counter()
dept_pair_counts = Counter()
for depts in order_dept_df['departments']:
    dept_counts.update(depts)
    dept_pair_counts.update(combinations(sorted(depts), 2))

In [10]:
dept_rules = []
for (dept1, dept2), count_pair in dept_pair_counts.items():
    support = count_pair / num_orders
    confidence_1_to_2 = count_pair / dept_counts[dept1]
    confidence_2_to_1 = count_pair / dept_counts[dept2]

    if support >= 0.003:
        if confidence_1_to_2 >= 0.25:
            dept_rules.append((dept1, dept2, support, confidence_1_to_2))
        if confidence_2_to_1 >= 0.25:
            dept_rules.append((dept2, dept1, support, confidence_2_to_1))

In [11]:
dept_rules_df = pd.DataFrame(dept_rules, columns=['department_id_1', 'department_id_2', 'support', 'confidence'])
dept_rules_df['department_name_1'] = dept_rules_df['department_id_1'].map(dept_id_to_name)
dept_rules_df['department_name_2'] = dept_rules_df['department_id_2'].map(dept_id_to_name)
dept_rules_df = dept_rules_df[['department_id_1', 'department_id_2', 'department_name_1', 'department_name_2', 'confidence', 'support']]
dept_rules_df.sort_values(by='department_id_1', ascending=False)

,department_id_1,department_id_2,department_name_1,department_name_2,confidence,support
115,21,4,missing,produce,0.850281,0.044928
131,21,13,missing,pantry,0.471225,0.024899
165,21,15,missing,canned goods,0.303765,0.016051
162,21,19,missing,snacks,0.559354,0.029556
161,21,3,missing,bakery,0.342709,0.018109
...,...,...,...,...,...,...
30,1,3,frozen,bakery,0.363416,0.141454
32,1,7,frozen,beverages,0.522939,0.203545
36,1,15,frozen,canned goods,0.298741,0.116280
38,1,16,frozen,dairy eggs,0.771534,0.300307


In [12]:
dept_rules_df.to_csv('departments_report.csv', index=False)

# Part B: Products report for each department

In [13]:
all_dept_rules = []
total_orders = len(order_df)
for dept_name in factors['department_name'].unique():
    dept_factors = factors[factors['department_name'] == dept_name]
    dept_orders = dept_factors.groupby('order_id')['product_id'].apply(list).reset_index(name='products')
    dept_orders = dept_orders[dept_orders['products'].apply(lambda x: len(set(x)) >= 2)]
    if dept_orders.empty:
        continue
        
    item_counts = Counter()
    pair_counts = Counter()

    for products in dept_orders['products']:
        unique_items = set(products)
        item_counts.update(unique_items)
        pair_counts.update(combinations(sorted(unique_items), 2))

    for (item1, item2), count_pair in pair_counts.items():
        support = count_pair / total_orders
        confidence_1_to_2 = count_pair / item_counts[item1]
        confidence_2_to_1 = count_pair / item_counts[item2]

        if support >= 0.003:
            if confidence_1_to_2 >= 0.25:
                all_dept_rules.append((dept_name, item1, item2, support, confidence_1_to_2))
            if confidence_2_to_1 >= 0.25:
                all_dept_rules.append((dept_name, item2, item1, support, confidence_2_to_1))

In [14]:
dept_rules_df = pd.DataFrame(all_dept_rules, columns=['department', 'product_id_1', 'product_id_2', 'confidence', 'support'])
dept_rules_df['product_name_1'] = dept_rules_df['product_id_1'].map(product_id_to_name)
dept_rules_df['product_name_2'] = dept_rules_df['product_id_2'].map(product_id_to_name)
dept_rules_df = dept_rules_df[['department', 'product_name_1', 'product_name_2', 'confidence', 'support']]
dept_rules_df.sort_values(['department', 'confidence'], ascending=[True, False])

,department,product_name_1,product_name_2,confidence,support
51,beverages,Lime Sparkling Water,Sparkling Water Grapefruit,0.003849,0.350451
2,produce,Organic Strawberries,Bag of Organic Bananas,0.023428,0.297321
0,produce,Organic Hass Avocado,Bag of Organic Bananas,0.018444,0.348201
17,produce,Organic Avocado,Banana,0.016889,0.314014
32,produce,Large Lemon,Banana,0.016447,0.277057
7,produce,Strawberries,Banana,0.014847,0.323212
1,produce,Organic Raspberries,Bag of Organic Bananas,0.013566,0.336994
21,produce,Organic Raspberries,Organic Strawberries,0.012728,0.316168
28,produce,Limes,Large Lemon,0.012156,0.275237
26,produce,Organic Blueberries,Organic Strawberries,0.009672,0.270460


In [15]:
dept_rules_df.to_csv('products_per_dept_report.csv', index=False)